In [2]:
import fsc
from fsc.devices import w7x
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import pyvista as pv

In [3]:
fsc.resolve.importOfflineData("../w7x.fsc")

In [4]:
geoGrid = w7x.defaultGeometryGrid
geometry = fsc.Geometry(fsc.data.readArchive('geometry.fsc'))

In [8]:
archiveCoils = fsc.data.readArchive("cadCoils.fsc")
config = w7x.standard(coils = archiveCoils)
grid = w7x.defaultGrid.clone()
grid.nR = 128
grid.nZ = 128
grid.nPhi = 32

In [9]:
tracer = fsc.tracer()

In [10]:
pLCFS = np.linspace([6.0, 0.0, 0.0], [6.3, 0, 0], 30, axis = 1)

traceResult = tracer.trace(pLCFS, config, geometry = geometry, grid = grid, geometryGrid = geoGrid, distanceLimit = 1e3, collisionLimit = 1)

In [11]:
iFirst = np.argmax(traceResult['stopReasons'] == 'collisionLimit')

In [12]:
surfacePoints = tracer.poincareInPhiPlanes(pLCFS[:,iFirst - 1], [0], 200, config, grid = grid)

In [13]:
startPoints = surfacePoints[:3,:,:]

tracingArgs = dict(
    isotropicDiffusionCoefficient = 1e-6,
    meanFreePath = 1,
    parallelConvectionVelocity = 1,
    grid = grid,
    geometryGrid = geoGrid,
    geometry = geometry,
    distanceLimit = 1e3,
    collisionLimit = 1,
    stepSize = 0.01
)

diffResult = tracer.trace(startPoints, config, **tracingArgs)

In [12]:
pvp = pv.Plotter()

resultPoints = diffResult['endPoints'][:3]
resultPoints = fsc.flt.symmetrize(resultPoints, 5, True)
pointCloud = pv.PolyData(resultPoints.reshape([3, -1]).T)

multiGeo = pv.MultiBlock([pointCloud, fsc.geometry.asPyvista(w7x.components([165], 'TDU'))])
pvp.add_mesh(multiGeo)
pvp.show(full_screen=True)

ViewInteractiveWidget(height=1200, layout=Layout(height='auto', width='100%'), width=1920)

In [9]:
pvp = pv.Plotter()

multiGeo = pv.MultiBlock([fsc.geometry.asPyvista(w7x.components([165], 'TDU'))])
pvp.add_mesh(multiGeo)
pvp.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)